#### Import

In [21]:
import os
import glob
import sys

import shutil
import json

import numpy as np
import random
import matplotlib.pyplot as plt
import scipy
from scipy import ndimage
import nibabel as nib
from scipy.ndimage import morphology
import SimpleITK
import pandas as pd

#### Test code of running imcut() remotely as we are in slicer 4.11 environment  where imcut() could not be implemented

In [22]:
# This flag will call graphcutServer in a different python environment within runAutoGCExperimentOnPatient
local = False 
verbose = False

In [23]:
#Configs
patDataConfig = \
{   
     "ctSuffix": "_ct.nii.gz",
     "ptSuffix": "_pt.nii.gz",
     "gtSuffix": "_ct_gtvt.nii.gz",
     "predSuffix": "_segment.nii.gz",
     "softmaxSuffix": "_softMax.nii.gz",
     "fgScribbleSuffix": "_fgScribble.nii.gz",
     "bgScribbleSuffix": "_bgScribble.nii.gz",
     "ct_gCutSuffix": "_ct_gCut.nii.gz",
     "pet_gCutSuffix": "_pet_gCut.nii.gz",         
     "softmax_gCutSuffix": "_softmax_gCut.nii.gz",
     "ct_low": -1000,
     "ct_high": 3095,
     "ct_clipImages": False,
     "ct_scaleImages": False,
     "ct_scaleFactor": 1000,
     "ct_normalizeImages": False,
     "pt_low": -1000,
     "pt_high": 3095,
     "pt_clipImages": False,
     "pt_scaleImages": False,
     "pt_scaleFactor": 1,
     "pt_normalizeImages": False,
     "labels_to_train": [1]
}


autoScribbleAndGCConfig = \
{
 'bbPad' : 2,
 'fractionDivider' : 10,
 'dilation_diam' : 2,
 'useAtmostNScribblesPerRegion' : 4,
 'segparams_ssgc' : 
        {
         "method": "graphcut",
         "pairwise_alpha": 1.0,
         "modelparams": 
            {
             "cvtype": "full",
             "params": 
                {
                 "covariance_type": "full",
                 "n_components": 1
                }
            },
         "return_only_object_with_seeds": True
        }     
}

### Helper functions 
#### Most of these has already been tested in other notebook(s); 

In [24]:
sys.path.append('/home/user/DMML/CodeAndRepositories/MMGTVSeg')
import src
from src import scribbleHelper, gcHelper, graphCutClient, graphCutServer, libclient, libserver
from src.scribbleHelper import checkFolderExistenceAndCreate, readAndScaleImageData,\
    dice_coef_func, dice_multi_label, disk, ball, bbox2_3D, getUnionBoundingBoxWithPadding,\
    chooseScribbleFromMissedFGOrWrongCBG2D, chooseScribbleFromDefiniteRegion2D,\
    autoGenerateScribbleRegions2D,\
    chooseScribbleFromMissedFGOrWrongCBG3D, chooseScribbleFromDefiniteRegion3D,\
    autoGenerateScribbleAndBBox3D,\
    createGCInputUsingGT
# import src.gcHelper as gcHelper

In [25]:
print('Running single experiment on individual patient')
srcFolder =\
'/home/user/DMML/Data/HeadNeck_PET_CT/nnUnet_3dfullres/validation_gtvs_withSoftmax'
expFolder = '/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment'
# srcFolder = 'J:/HecktorData/nnUnet_3dfullres/validation_gtvs_withSoftmax'
# expFolder = 'J:/PlayDataManualSegmentation/AutoScribbleExperiment'
patientName = 'CHUM038'
ctData,  ptData, gtData, softmaxData, predFromNN,\
    binLimit, bbVolume, numFGS, numBGS,\
    fgScribbleFromFGMissed, bgScribbleFromBGWrongC,\
    fgScribbleFromDefiniteFG, bgScribbleFromDefiniteBG,\
    fgScribble, bgScribble,\
    graphCutInputConfig, graphCutInputConfig_JsonFilePath\
   =  createGCInputUsingGT(patientName=patientName, srcFolder=srcFolder,\
        expFolder=expFolder, expPatName = "expPat",\
        patDataConfig=patDataConfig,\
        autoScribbleAndGCConfig = autoScribbleAndGCConfig,\
        verbose=verbose)

from src.graphCutClient import sendImCutRqstAndReceiveResult
resultAvailable, gcAndDiceResult = \
    sendImCutRqstAndReceiveResult(graphCutInputConfig_JsonFilePath)
print(gcAndDiceResult)

Running single experiment on individual patient
Starting action :  imcut
starting connection to ('127.0.0.1', 65432)
sending b'\x00h{"byteorder": "little", "content-type": "text/json", "content-encoding": "utf-8", "content-length": 127}{"action": "imcut", "value": "/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/graphCutInputConfig.json"}' to ('127.0.0.1', 65432)
received text/json response from ('127.0.0.1', 65432)
graphCut success : True
closing connection to ('127.0.0.1', 65432)
{'successFlag': True, 'patientName': 'CHUM038', 'gcPath_ct': '/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/expPat_ct_gCut.nii.gz', 'gcPath_pet': '/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/expPat_pet_gCut.nii.gz', 'gcPath_softmax': '/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/expPat_softmax_gCut.nii.gz', 'originalDice': 0.48517890685847803, 'numFGS': 8, 'numBGS': 8, 'gcDice_ct': 0.37874897039585254, 'gcDic

In [26]:
#from src import graphCutClient 
def runAutoGCExperimentOnPatient(patientName, srcFolder, expFolder,\
                                 patDataConfig, autoScribbleAndGCConfig, 
                                 numExperimentsPerPat, verbose=False, local=True):
    
    experimentResultDetail = []
    for expId in range(numExperimentsPerPat):
        ctData,  ptData, gtData, softmaxData, predFromNN,\
            binLimit, bbVolume, numFGS, numBGS,\
            fgScribbleFromFGMissed, bgScribbleFromBGWrongC,\
            fgScribbleFromDefiniteFG, bgScribbleFromDefiniteBG,\
            fgScribble, bgScribble,\
            graphCutInputConfig, graphCutInputConfig_JsonFilePath\
           =  createGCInputUsingGT(patientName=patientName, srcFolder=srcFolder,\
                expFolder=expFolder, expPatName = "expPat",\
                patDataConfig=patDataConfig,\
                autoScribbleAndGCConfig = autoScribbleAndGCConfig,\
                verbose=verbose)
        if True == local:
            from src.gcHelper import local_generateGrahcutSegmentationAndDiceFromJson    
            gcAndDiceResult = local_generateGrahcutSegmentationAndDiceFromJson(graphCutInputConfig_JsonFilePath)
        else:
            from src.gcHelper import remote_generateGrahcutSegmentationAndDiceFromJson    
            gcAndDiceResult = remote_generateGrahcutSegmentationAndDiceFromJson(graphCutInputConfig_JsonFilePath)
#             from src.graphCutClient import sendImCutRqstAndReceiveResult
#             resultAvailable, gcAndDiceResult = \
#                 sendImCutRqstAndReceiveResult(graphCutInputConfig_JsonFilePath)
        experimentResultDetail.append([gcAndDiceResult["patientName"], gcAndDiceResult["successFlag"],\
         gcAndDiceResult["numFGS"], gcAndDiceResult["numBGS"],\
         gcAndDiceResult["originalDice"], gcAndDiceResult["gcDice_softmax"],\
         gcAndDiceResult["gcDice_ct"], gcAndDiceResult["gcDice_pet"]])
    # Create the pandas DataFrame 
    expResult_df = pd.DataFrame(experimentResultDetail,\
         columns = ['patientName', 'successFlag', '#FGScrb', '#BGScrb', 'd_org', ' d_softmax', ' d_ct', ' d_pet'])
    return expResult_df

In [27]:
print('Running multiple experiment on individual patient')
srcFolder =\
'/home/user/DMML/Data/HeadNeck_PET_CT/nnUnet_3dfullres/validation_gtvs_withSoftmax'
expFolder = '/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment'
# srcFolder = 'J:/HecktorData/nnUnet_3dfullres/validation_gtvs_withSoftmax'
# expFolder = 'J:/PlayDataManualSegmentation/AutoScribbleExperiment'
patientName = 'CHUM038'
numExperimentsPerPat=5    
expResult_df = runAutoGCExperimentOnPatient(patientName, srcFolder, expFolder,\
    patDataConfig, autoScribbleAndGCConfig,\
    numExperimentsPerPat, verbose, local)
print(expResult_df)

Running multiple experiment on individual patient
Starting action :  imcut
starting connection to ('127.0.0.1', 65432)
sending b'\x00h{"byteorder": "little", "content-type": "text/json", "content-encoding": "utf-8", "content-length": 127}{"action": "imcut", "value": "/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/graphCutInputConfig.json"}' to ('127.0.0.1', 65432)
received text/json response from ('127.0.0.1', 65432)
graphCut success : True
closing connection to ('127.0.0.1', 65432)
Starting action :  imcut
starting connection to ('127.0.0.1', 65432)
sending b'\x00h{"byteorder": "little", "content-type": "text/json", "content-encoding": "utf-8", "content-length": 127}{"action": "imcut", "value": "/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/graphCutInputConfig.json"}' to ('127.0.0.1', 65432)
received text/json response from ('127.0.0.1', 65432)
graphCut success : True
closing connection to ('127.0.0.1', 65432)
Starting action :  imcut
s

### Interactive scribble and graphcut Plan

In [28]:
import JupyterNotebooksLib as slicernb
slicernb.ViewDisplay('OneUpRedSlice') #'FourUp', 'OneUp3D'
slicer.mrmlScene.Clear()

#ctVolumeNode = slicer.util.addVolumeFromArray(ctData,ijkToRAS=None,name='ctVolume',nodeClassName='vtkMRMLScalarVolumeNode')
ctVolumeNode = slicer.util.loadVolume(os.path.join(expFolder,ctFileName))
ptVolumeNode = slicer.util.loadVolume(os.path.join(expFolder,ptFileName))
softmaxVolumeNode = slicer.util.loadVolume(os.path.join(expFolder,softmaxFileName))

def createSegmentationNodeFromLabelFile(folderPath, labelFileName, labelMapName, segmentationNodeName, colorTriple, sliceFillFlag):
    #labelMapNode = slicer.util.addVolumeFromArray(labelData,ijkToRAS=None,name=labelMapName,nodeClassName='vtkMRMLScalarVolumeNode')
    labelMapNode = slicer.util.loadLabelVolume(os.path.join(folderPath, labelFileName), properties = {'name': labelMapName})
    segmentationNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode", segmentationNodeName)
    slicer.modules.segmentations.logic().ImportLabelmapToSegmentationNode(labelMapNode, segmentationNode)
    slicer.mrmlScene.RemoveNode(labelMapNode)
    segmentationNode.GetSegmentation().GetSegment(labelMapName).SetColor(colorTriple[0], colorTriple[1],colorTriple[2])
    if True == sliceFillFlag:
        segmentationNode.GetDisplayNode().SetVisibility2DFill(1)
    else:
        segmentationNode.GetDisplayNode().SetVisibility2DFill(0)
    return segmentationNode

gt_segmentationNode = createSegmentationNodeFromLabelFile(expFolder, gtFileName, \
                         "gtLabelMap", "gtSeg", (0,0,1), False)
pred_segmentationNode = createSegmentationNodeFromLabelFile(expFolder, predFileName, \
                         "predLabelMap", "predSeg", (1,0,0), False)
fgSeeds_segmentationNode = createSegmentationNodeFromLabelFile(expFolder, fgSeedsFileName, \
                         "fgSeedsLabelMap", "fgSeedsSeg", (0.15, 0.7, 0.8), True)
bgSeeds_segmentationNode = createSegmentationNodeFromLabelFile(expFolder, bgSeedsFileName, \
                         "bgSeedsLabelMap", "bgSeedsSeg", (0.8, 0.7, 0.15), True)

# slicer.util.setSliceViewerLayers(background=ctVolumeNode, foreground=ptVolumeNode, label=gt_segmentationNode\
#                                 foregroundOpacity=0.5,labelOpacity=0.5)
slicer.util.setSliceViewerLayers(background=softmaxVolumeNode, foreground=ptVolumeNode)
getNode('expPat_pt').GetDisplayNode().SetAndObserveColorNodeID('vtkMRMLPETProceduralColorNodePET-Heat')

NameError: name 'ctFileName' is not defined

In [ ]:
slicer.mrmlScene.Clear()

In [ ]:
#Python interactor output
# >>> expPat_pt_VolumeNode = getNode('expPat_pt')
# >>> type(expPat_pt_VolumeNode)
# <class 'MRMLCorePython.vtkMRMLScalarVolumeNode'>
# >>> expPat_pt_DisplayNode = expPat_pt_VolumeNode.GetDisplayNode()
# >>> type(expPat_pt_DisplayNode)
# <class 'MRMLCorePython.vtkMRMLScalarVolumeDisplayNode'>
# >>> expPat_pt_ColorNode = expPat_pt_DisplayNode.GetColorNode()
# >>> type(expPat_pt_ColorNode)
# <class 'MRMLCorePython.vtkMRMLColorTableNode'>
# >>> expPat_pt_CurrentLUT = expPat_pt_ColorNode.GetLookupTable()
# >>> type(expPat_pt_CurrentLUT)
# <class 'vtkCommonCorePython.vtkLookupTable'>

# >>> indexL = expPat_pt_CurrentLUT.GetIndexedLookup()
# >>> type(indexL)
# <class 'int'>
# >>> print(indexL)
# 0
# >>> expPat_pt_CurrentLUT.SetIndexedLookup(3)
# >>> expPat_pt_CurrentLUT = expPat_pt_ColorNode.GetLookupTable()
# >>> print(expPat_pt_CurrentLUT.GetIndexedLookup())
# 3
# >>> getNode('expPat_pt').GetDisplayNode().GetColorNode().IsTypeOf('vtkMRMLColorTableNode')
# 1
# >>> getNode('expPat_pt').GetDisplayNode().GetColorNode().IsTypeOf('vtkMRMLProceduralColorNode')
# 0

In [ ]:
#Experimental read write json file
with open("/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/expPat_graphCutInput.json") as fp:
        gcSConfig = json.load(fp)
        fp.close() 
srcFolder = gcSConfig["srcFolder" ]
patDataConfig = gcSConfig["patDataConfig" ]
autoScribbleAndGCConfig = gcSConfig['autoScribbleAndGCConfig']
expFolder = patDataConfig["expFolder"]

print(srcFolder)
print(expFolder)
print(autoScribbleAndGCConfig)
print(patDataConfig['labels_to_train'])

#Make changes

new_expFolder = "/home/user/DMML/Data/PlayDataManualSegmentation/ManualScribble"
new_patDataConfig = patDataConfig
new_patDataConfig["expFolder"]= new_expFolder

new_srcFolder = "/home/user/DMML/Data/HeadNeck_PET_CT/nnUnet_3dfullres/training_gtvs_withSoftmax"
new_gcSConfig = gcSConfig
new_gcSConfig["srcFolder" ] = new_srcFolder
new_gcSConfig["patDataConfig" ] = new_patDataConfig
with open("/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/modified_expPat_graphCutInput.json", 'w') as fp:
        json.dump(new_gcSConfig, fp, ) #, indent='' #, indent=4
        fp.close()
        
#Let us test
#Experimental read write json file
with open("/home/user/DMML/Data/PlayDataManualSegmentation/AutoScribbleExperiment/modified_expPat_graphCutInput.json") as fp:
        gcSConfig2 = json.load(fp)
        fp.close() 
srcFolder2 = gcSConfig2["srcFolder" ]
patDataConfig2 = gcSConfig2["patDataConfig" ]
autoScribbleAndGCConfig2 = gcSConfig2['autoScribbleAndGCConfig']
expFolder2 = patDataConfig2["expFolder"]

print('####################')
print(srcFolder2)
print(expFolder2)
print(autoScribbleAndGCConfig2)
print(patDataConfig2['labels_to_train'])